**Bài 1**

In [14]:
import re

# Hàm để lấy dữ liệu từ file input
def get_data_from_file():
    with open("SampleInput.txt", 'r') as file:
        return file.read()

# Hàm để chuyển đổi dữ liệu sang định dạng yêu cầu
def convert_data(data):
    # Tìm tất cả các cặp dữ liệu Grade và các dữ liệu liên quan
    grade_data = re.findall(r'Grade = (\d+)(.*?)\n\n\n|$', data, re.DOTALL)
    # print(grade_data)
    
    result = []
    grade_data = grade_data[:-2]
    
    for grade_match in grade_data:
        grade = grade_match[0]
        students = re.findall(r'\n(\d+), (\w+)', grade_match[1], re.MULTILINE)

        len_ = len(students)
        
        for i in range(len_ // 2):
            result.append((grade, students[i][0], students[i][1], students[i + len_ // 2][1]))
    
    return result

# Hàm để in ra dữ liệu theo định dạng bảng
def print_data(data, school_data):
    print("School\t\t\tGrade\tStudent number\t\tName\t\t\tScore")
    current_grade = ''
    for grade, number, name, score in data:
        if grade:
            current_grade = grade
        print("{:<23}\t{:<10}\t{:<15}\t{:<20}\t{}".format(school_data, current_grade, number, name, score))

# Đường dẫn tới file input
file_path = "SampleInput.txt"

# Lấy dữ liệu từ file
input_data = get_data_from_file()

# Chuyển đổi dữ liệu
converted_data = convert_data(input_data)

school_data = re.findall(r'School = (.*?)\n|$', input_data, re.DOTALL)

# In ra dữ liệu theo định dạng bảng
print_data(converted_data, school_data[0])


School			Grade	Student number		Name			Score
Riverdale High         	1         	0              	Phoebe              	3
Riverdale High         	1         	1              	Rachel              	7
Riverdale High         	2         	0              	Angela              	6
Riverdale High         	2         	1              	Tristan             	3
Riverdale High         	2         	2              	Aurora              	9


**Bài 3**

In [15]:
import pandas as pd
from bs4 import BeautifulSoup

In [16]:
df = pd.read_csv("labeledTrainData.xlsx.tsv", sep='\t')

df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


3.1  Loại bỏ các thẻ HTML

In [17]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

In [18]:
print(df['review'].iloc[9])
df['review'] = df['review'].apply(remove_html_tags)
print(df['review'].iloc[9])

<br /><br />This movie is full of references. Like \Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."


C:\Users\trinh\AppData\Local\Temp\ipykernel_9292\2125600460.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


This movie is full of references. Like \Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."


3.2  Loại bỏ các non-letters 

In [19]:
import re

def remove_non_letter(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [20]:
print("Before remove non-letter: ")
print(df['review'].iloc[9])
df['review'] = df['review'].apply(remove_non_letter)
print("After remove non-letter: ")
print(df['review'].iloc[9])

Before remove non-letter: 
This movie is full of references. Like \Mad Max II\", \"The wild one\" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpiece. We´ll talk much more about in the future."
After remove non-letter: 
This movie is full of references Like Mad Max II The wild one and many others The ladybugs face its a clear reference or tribute to Peter Lorre This movie is a masterpiece Well talk much more about in the future


3.3  Chuyển các từ thành chữ thường và tách thành các từ riêng biệt với giả thuyết là các 
từ cách nhau bởi khoảng trắng

In [21]:
def lower_and_split(text):
    lowercase_text = text.lower()
    words = lowercase_text.split()
    return words

In [22]:
print("Before: ")
print(df['review'].iloc[9])
df['review'] = df['review'].apply(lower_and_split)
print("After: ")
print(df['review'].iloc[9])

Before: 
This movie is full of references Like Mad Max II The wild one and many others The ladybugs face its a clear reference or tribute to Peter Lorre This movie is a masterpiece Well talk much more about in the future
After: 
['this', 'movie', 'is', 'full', 'of', 'references', 'like', 'mad', 'max', 'ii', 'the', 'wild', 'one', 'and', 'many', 'others', 'the', 'ladybugs', 'face', 'its', 'a', 'clear', 'reference', 'or', 'tribute', 'to', 'peter', 'lorre', 'this', 'movie', 'is', 'a', 'masterpiece', 'well', 'talk', 'much', 'more', 'about', 'in', 'the', 'future']


3.4   Loại bỏ các stop words

In [23]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trinh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trinh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
from nltk.corpus import stopwords

In [25]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    
    words = text
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # filtered_text = ' '.join(filtered_words)
    
    return filtered_words

In [26]:
print("Before: ")
print(df['review'].iloc[9])
df['review'] = df['review'].apply(remove_stopwords)
print("After: ")
print(df['review'].iloc[9])

Before: 
['this', 'movie', 'is', 'full', 'of', 'references', 'like', 'mad', 'max', 'ii', 'the', 'wild', 'one', 'and', 'many', 'others', 'the', 'ladybugs', 'face', 'its', 'a', 'clear', 'reference', 'or', 'tribute', 'to', 'peter', 'lorre', 'this', 'movie', 'is', 'a', 'masterpiece', 'well', 'talk', 'much', 'more', 'about', 'in', 'the', 'future']
After: 
['movie', 'full', 'references', 'like', 'mad', 'max', 'ii', 'wild', 'one', 'many', 'others', 'ladybugs', 'face', 'clear', 'reference', 'tribute', 'peter', 'lorre', 'movie', 'masterpiece', 'well', 'talk', 'much', 'future']


3.5   Chuyển các từ về từ gốc: Stem words (sử dụng thư viện SnowballStemmer của nltk)

In [27]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

In [28]:
def stem_words(text):
    stemmer = SnowballStemmer(language='english')
    text = ' '.join(text)
    words = word_tokenize(text)
    
    stemmed_words = [stemmer.stem(word) for word in words]
    
    # stemmed_text = ' '.join(stemmed_words)
    
    return stemmed_words

In [29]:
print("Before: ")
print(df['review'].iloc[9])
df['review'] = df['review'].apply(stem_words)
print("After: ")
print(df['review'].iloc[9])

Before: 
['movie', 'full', 'references', 'like', 'mad', 'max', 'ii', 'wild', 'one', 'many', 'others', 'ladybugs', 'face', 'clear', 'reference', 'tribute', 'peter', 'lorre', 'movie', 'masterpiece', 'well', 'talk', 'much', 'future']


After: 
['movi', 'full', 'refer', 'like', 'mad', 'max', 'ii', 'wild', 'one', 'mani', 'other', 'ladybug', 'face', 'clear', 'refer', 'tribut', 'peter', 'lorr', 'movi', 'masterpiec', 'well', 'talk', 'much', 'futur']


3.6   Nối các từ thành một chuỗi, các từ cách nhau bởi khoảng trắng và trả về kết quả 

In [30]:
def join_words(words):
    result = ' '.join(words)
    return result

In [31]:
print("Before: ")
print(df['review'].iloc[9])
df['review'] = df['review'].apply(join_words)
print("After: ")
print(df['review'].iloc[9])

Before: 
['movi', 'full', 'refer', 'like', 'mad', 'max', 'ii', 'wild', 'one', 'mani', 'other', 'ladybug', 'face', 'clear', 'refer', 'tribut', 'peter', 'lorr', 'movi', 'masterpiec', 'well', 'talk', 'much', 'futur']
After: 
movi full refer like mad max ii wild one mani other ladybug face clear refer tribut peter lorr movi masterpiec well talk much futur


3.7   Chuyển thành đặc trưng TF-IDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(df['review'])

print("Shape of TF-IDF matrix:", tfidf_matrix.shape)

print("Vocabulary:", tfidf_vectorizer.get_feature_names_out())

print("TF-IDF matrix:")
print(tfidf_matrix.toarray())

Shape of TF-IDF matrix: (25000, 112296)
Vocabulary: ['00' '000' '0000000000001' ... 'zzzzz' 'zzzzzzzz'
 'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz']
TF-IDF matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


3.8   Tính độ đo tương đồng Cosin dựa trên đặc trưng TF-IDF của 2 câu bất kỳ

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([df['review'].iloc[8], df['review'].iloc[9]])
cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

In [ ]:
print("Cosine similarity between the two sentences:", cosine_sim[0][0])

Cosine similarity between the two sentences: 0.008417265597487362
